In [ ]:
# ✅ Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# ✅ Dataset path
DATASET_DIR = '/content/drive/MyDrive/dataset_cancer_v1/classificacao_binaria/100X'

# ✅ Imports
import os
import numpy as np
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization, Activation, Add, GlobalAveragePooling2D, DepthwiseConv2D
from tensorflow.keras.optimizers import Adam
import random

# === CONFIG ===
IMG_SIZE = (224, 224)  # Standard for EfficientNet
EPOCHS = 10
BATCH_SIZE = 32
LEARNING_RATE = 0.0001

In [ ]:
# === EfficientNet Building Blocks ===
def swish_activation(x):
    """Swish activation function"""
    return x * Activation('sigmoid')(x)

def mb_conv_block(x, filters, kernel_size, stride, expand_ratio, se_ratio, name):
    """Mobile Inverted Residual Block (MBConv)"""
    input_channels = x.shape[-1]
    expanded_channels = int(input_channels * expand_ratio)
    
    # Expansion phase
    if expand_ratio != 1:
        x = Conv2D(expanded_channels, 1, padding='same', use_bias=False, name=f'{name}_expand')(x)
        x = BatchNormalization(name=f'{name}_expand_bn')(x)
        x = Activation(swish_activation, name=f'{name}_expand_swish')(x)
    
    # Depthwise convolution
    x = DepthwiseConv2D(kernel_size, strides=stride, padding='same', use_bias=False, name=f'{name}_depthwise')(x)
    x = BatchNormalization(name=f'{name}_depthwise_bn')(x)
    x = Activation(swish_activation, name=f'{name}_depthwise_swish')(x)
    
    # Squeeze-and-Excitation
    if se_ratio > 0:
        se_channels = max(1, int(expanded_channels * se_ratio))
        se = GlobalAveragePooling2D(name=f'{name}_se_gap')(x)
        se = Dense(se_channels, activation='relu', name=f'{name}_se_fc1')(se)
        se = Dense(expanded_channels, activation='sigmoid', name=f'{name}_se_fc2')(se)
        se = Conv2D(expanded_channels, 1, padding='same', name=f'{name}_se_conv')(se)
        x = Add(name=f'{name}_se_add')([x, se])
    
    # Output projection
    x = Conv2D(filters, 1, padding='same', use_bias=False, name=f'{name}_project')(x)
    x = BatchNormalization(name=f'{name}_project_bn')(x)
    
    # Residual connection
    if stride == 1 and input_channels == filters:
        x = Add(name=f'{name}_residual')([x, x])
    
    return x

In [ ]:
# === Visualize Sample Images ===
def plot_sample_images(dataset_path, num_images_per_class=5):
    classes = sorted(os.listdir(dataset_path))
    # plt.figure(figsize=(15, len(classes) * 2.5))
    for class_idx, class_name in enumerate(classes):
        class_path = os.path.join(dataset_path, class_name)
        images = [img for img in os.listdir(class_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
        sample_images = random.sample(images, min(num_images_per_class, len(images)))
        for i, image_name in enumerate(sample_images):
            img_path = os.path.join(class_path, image_name)
            img = load_img(img_path, target_size=IMG_SIZE)
            # plt.subplot(len(classes), num_images_per_class, class_idx * num_images_per_class + i + 1)
            # plt.imshow(img)
            # plt.axis('off')
            # plt.title(class_name)
    # plt.tight_layout()
    # plt.show()

# === Load Dataset ===
def load_dataset(path):
    images = []
    labels = []
    for label_name in os.listdir(path):
        label_folder = os.path.join(path, label_name)
        if os.path.isdir(label_folder):
            for fname in os.listdir(label_folder):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(label_folder, fname)
                    img = load_img(img_path, target_size=IMG_SIZE)
                    img = img_to_array(img) / 255.0
                    images.append(img)
                    labels.append(label_name)
    return np.array(images), np.array(labels)

In [ ]:
# === Load and Prepare Data ===
print("🔄 Loading dataset...")
X, y = load_dataset(DATASET_DIR)
print(f"✅ Loaded {len(X)} images.")

# === Visualize ===
print("🖼 Displaying sample images...")
plot_sample_images(DATASET_DIR)

In [ ]:
# === Encode Labels ===
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
loss_fn = 'binary_crossentropy'
activation_fn = 'sigmoid'
output_units = 1

# === Split Dataset ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

In [ ]:
# === Build EfficientNet Model ===
def build_efficientnet_model(input_shape=(224, 224, 3), num_classes=1):
    inputs = Input(shape=input_shape)
    
    # Initial convolution
    x = Conv2D(32, 3, strides=2, padding='same', use_bias=False, name='conv1')(inputs)
    x = BatchNormalization(name='bn1')(x)
    x = Activation(swish_activation, name='swish1')(x)
    
    # MBConv blocks (simplified EfficientNet-B0 architecture)
    # Block 1
    x = mb_conv_block(x, 16, 3, 1, 1, 0.25, 'block1')
    
    # Block 2
    x = mb_conv_block(x, 24, 3, 2, 6, 0.25, 'block2')
    x = mb_conv_block(x, 24, 3, 1, 6, 0.25, 'block2_2')
    
    # Block 3
    x = mb_conv_block(x, 40, 5, 2, 6, 0.25, 'block3')
    x = mb_conv_block(x, 40, 5, 1, 6, 0.25, 'block3_2')
    
    # Block 4
    x = mb_conv_block(x, 80, 3, 2, 6, 0.25, 'block4')
    x = mb_conv_block(x, 80, 3, 1, 6, 0.25, 'block4_2')
    x = mb_conv_block(x, 80, 3, 1, 6, 0.25, 'block4_3')
    
    # Block 5
    x = mb_conv_block(x, 112, 5, 1, 6, 0.25, 'block5')
    x = mb_conv_block(x, 112, 5, 1, 6, 0.25, 'block5_2')
    x = mb_conv_block(x, 112, 5, 1, 6, 0.25, 'block5_3')
    
    # Block 6
    x = mb_conv_block(x, 192, 5, 2, 6, 0.25, 'block6')
    x = mb_conv_block(x, 192, 5, 1, 6, 0.25, 'block6_2')
    x = mb_conv_block(x, 192, 5, 1, 6, 0.25, 'block6_3')
    x = mb_conv_block(x, 192, 5, 1, 6, 0.25, 'block6_4')
    
    # Block 7
    x = mb_conv_block(x, 320, 3, 1, 6, 0.25, 'block7')
    
    # Final convolution
    x = Conv2D(1280, 1, padding='same', use_bias=False, name='conv_final')(x)
    x = BatchNormalization(name='bn_final')(x)
    x = Activation(swish_activation, name='swish_final')(x)
    
    # Global average pooling
    x = GlobalAveragePooling2D(name='gap')(x)
    
    # Fully connected layers
    x = Dense(512, activation='relu', name='fc1')(x)
    x = Dropout(0.5, name='dropout1')(x)
    x = Dense(256, activation='relu', name='fc2')(x)
    x = Dropout(0.5, name='dropout2')(x)
    outputs = Dense(num_classes, activation=activation_fn, name='predictions')(x)
    
    model = Model(inputs, outputs, name='EfficientNet')
    return model

# Create model
model = build_efficientnet_model(input_shape=(*IMG_SIZE, 3), num_classes=output_units)

# Compile
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss=loss_fn, metrics=['accuracy'])
model.summary()

In [ ]:
# === Train Model ===
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)

In [ ]:
# === Evaluate ===
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"📊 Test Accuracy: {test_acc:.2f}")

In [ ]:
# === Prediction Function ===
def predict_image(img_path):
    img = load_img(img_path, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    label = "Malignant" if prediction[0][0] > 0.5 else "Benign"
    confidence = prediction[0][0] if prediction[0][0] > 0.5 else 1 - prediction[0][0]

    print(f"Prediction: {label} ({confidence:.2f})")
    # plt.imshow(load_img(img_path))
    # plt.title(f"{label} ({confidence:.2f})")
    # plt.axis('off')
    # plt.show()

# Example:
# predict_image('/content/drive/MyDrive/dataset_cancer_v1/classificacao_binaria/100X/benign/img001.png')